In [1]:

%pprint
%run ../load_magic/storage.py
%run ../load_magic/lists.py
import collections

s = Storage()

Pretty printing has been turned OFF


In [2]:

def get_proportion_score(word_str, letter_proportions_df):
    word_score = 0
    for row_index, row_series in letter_proportions_df.iterrows():
        letter = row_series.letter_char
        if letter in word_str:
            proportion = row_series.proportion
            word_score += proportion
    
    return word_score

In [3]:

def get_best_word(candidates_list):
    
    # Get proportions from candidate list
    import collections

    letters_list = []
    for word_str in candidates_list:
        letters_list += list(word_str)
    counter = collections.Counter(letters_list)
    letter_proportions_df = pd.DataFrame([{'letter_char': x, 'letter_count': y} for x, y in dict(counter).items()])
    min_count = letter_proportions_df.letter_count.min()
    letter_proportions_df['proportion'] = letter_proportions_df.letter_count.map(lambda x: x/min_count)
    
    # Minimize words to pick through after you get the next word colors back
    
    # Calculate maximum proportionality
    max_score = 0
    max_word = ''
    for word_str in candidates_list:
        word_score = get_proportion_score(word_str, letter_proportions_df)
        if word_score > max_score:
            max_score = word_score
            max_word = word_str
    
    return max_word, max_score

In [4]:

def measure_word(test_word, target_word):
    colors_list = []
    for i in range(5):
        if test_word[i] == target_word[i]:
            colors_list.append('G')
        elif test_word[i] in target_word:
            colors_list.append('Y')
        else:
            colors_list.append('x')
    
    return ''.join(colors_list)

In [5]:

def get_word_guess(response_patterns_df, candidates_list, guesses_list=[]):
    if len(candidates_list) == 1:
        
        return candidates_list[0]
    mask_series = response_patterns_df.target_word.isin(candidates_list) & ~response_patterns_df.test_word.isin(guesses_list)
    columns_list = ['test_word', 'response_pattern']
    guess_df = response_patterns_df[mask_series].groupby(columns_list).count()
    guess_df = guess_df.reset_index().groupby('test_word').max().sort_values(by='target_word').head(20)

    # Get proportions from candidate list
    if guess_df.shape[0] > 1:
        letters_list = []
        for word_str in guess_df.index:
            letters_list += list(word_str)
        counter = collections.Counter(letters_list)
        letter_proportions_df = pd.DataFrame([{'letter_char': x, 'letter_count': y} for x, y in dict(counter).items()])
        min_count = letter_proportions_df.letter_count.min()
        letter_proportions_df['proportion'] = letter_proportions_df.letter_count.map(lambda x: x/min_count)

        guess_df['word_score'] = guess_df.index.map(lambda x: get_proportion_score(x, letter_proportions_df))
        max_word_score = guess_df.word_score.max()
        mask_series = (guess_df.word_score == max_word_score)
        guess_df = guess_df[mask_series]
    
    guesses_list = guess_df.sort_values(by='target_word', ascending=False).index.tolist()
    guess = None
    if guesses_list:
        guess = guesses_list[0]
    
    return guess

In [6]:

def get_greens_set(test_words_list, response_patterns_list):
    greens_set = set()
    for test_word, response_pattern in zip(test_words_list, response_patterns_list):
        for i in range(5):
            if response_pattern[i] == 'G':
                greens_set.add(test_word[i])
    
    return greens_set

In [7]:

def get_yellows_set(test_words_list, response_patterns_list):
    yellows_set = set()
    for test_word, response_pattern in zip(test_words_list, response_patterns_list):
        for i in range(5):
            if response_pattern[i] == 'Y':
                yellows_set.add(test_word[i])
    
    return yellows_set

In [8]:

def get_greys_list(test_words_list, response_patterns_list):
    greens_set = get_greens_set(test_words_list, response_patterns_list)    
    yellows_set = get_yellows_set(test_words_list, response_patterns_list)
    greys_set = set()
    for test_word, response_pattern in zip(test_words_list, response_patterns_list):
        for i in range(5):
            if (response_pattern[i] == 'x') and (test_word[i] not in greens_set) and (test_word[i] not in yellows_set):
                greys_set.add(test_word[i])
    greys_list = sorted(greys_set)
    
    return greys_list

In [9]:

def get_yellows_list(test_words_list, response_patterns_list):
    yellows_set = set()
    for test_word, response_pattern in zip(test_words_list, response_patterns_list):
        for i in range(5):
            if (response_pattern[i] == 'Y'):
                yellows_set.add(test_word[i])
    yellows_list = sorted(yellows_set)
    
    return yellows_list

In [10]:

def is_greened(word_str, test_words_list, response_patterns_list):
    transposed_tests_list = list(map(list, zip(*test_words_list)))
    transposed_responses_list = list(map(list, zip(*response_patterns_list)))
    is_greened = True
    for i in range(5):
        test_chars_list = transposed_tests_list[i]
        response_chars_list = transposed_responses_list[i]
        if 'G' in response_chars_list:
            idx = response_chars_list.index('G')
            is_greened = is_greened and (word_str[i] == test_chars_list[idx])
        else:
            is_greened = is_greened and (word_str[i] not in test_chars_list)
    
    return is_greened

In [11]:

def get_candidates_list(test_words_list, response_patterns_list, previous_candidates_list=[]):
    mask_series = False
    for test_word, response_pattern in zip(test_words_list, response_patterns_list):
        mask_series = mask_series | ((response_patterns_df.test_word == test_word) & (response_patterns_df.response_pattern == response_pattern))
    candidates_df = response_patterns_df[mask_series].groupby('target_word').count()
    mask_series = (candidates_df.response_pattern == len(response_patterns_list))
    candidates_list = sorted(set(candidates_df[mask_series].index.tolist()))
    if previous_candidates_list and not candidates_list:
        greys_list = get_greys_list(test_words_list, response_patterns_list)
        yellows_list = get_yellows_list(test_words_list, response_patterns_list)
        for word_str in previous_candidates_list:
            if all(map(lambda x: x in word_str, yellows_list)):
                if all(map(lambda x: x not in word_str, greys_list)):
                    if is_greened(word_str, test_words_list, response_patterns_list):
                        candidates_list.append(word_str)
    
    return candidates_list

In [12]:

# Get words list
if s.pickle_exists('wordle_words_list'):
    words_list = s.load_object('wordle_words_list')
else:
    
    # Get words from the Wordle JavaScript
    import re

    words_list = []
    az_regex = re.compile('[^a-z]')
    file_path = r'D:\Documents\GitHub\notebooks\data\txt\wordle_words_short_list.txt'
    with open(file_path, encoding='utf8') as infile:
        for line in infile:
            word_str = line.strip()
            if (len(word_str) == 5) and not az_regex.search(word_str):
                words_list.append(word_str)
    file_path = r'D:\Documents\GitHub\notebooks\data\txt\wordle_words_long_list.txt'
    with open(file_path, encoding='utf8') as infile:
        for line in infile:
            word_str = line.strip()
            if (len(word_str) == 5) and not az_regex.search(word_str):
                words_list.append(word_str)
    words_list = list(set(words_list))
    # import requests
    
    # link = 'https://www.powerlanguage.co.uk/wordle/main.814b2d17.js'
    # f = requests.get(link)
    # commands_list = f.text.split(';')
    # list_str = commands_list[331].split(']')[0].split('[')[1]
    # words_list = eval(f'[{list_str}]')
    s.store_objects(wordle_words_list=words_list)

In [13]:

# Get the response patterns
if s.pickle_exists('wordle_response_patterns_df'):
    response_patterns_df = s.load_object('wordle_response_patterns_df')
else:
    # rows_list = []
    # for test_word in words_list:
    #     for target_word in words_list:
    #         row_dict = {}
    #         row_dict['test_word'] = test_word
    #         row_dict['target_word'] = target_word
    #         row_dict['response_pattern'] = measure_word(test_word, target_word)
    #         rows_list.append(row_dict)
    # response_patterns_df = pd.DataFrame(rows_list)
    response_patterns_df = s.load_csv('response_patterns_df', folder_path=s.saves_folder)
    s.store_objects(wordle_response_patterns_df=response_patterns_df)


----
# Minimize Branches to Solve a Wordle

In [14]:

tree_first_guess = get_word_guess(response_patterns_df, words_list)
print(f'list of possible solutions: {len(words_list):,}')
print(f'FIRST MOVE: {tree_first_guess}')

list of possible solutions: 12,972
FIRST MOVE: reals


In [23]:

tree_first_guess = 'reals'
tree_first_response_pattern = 'xYxxx'
tree_first_candidates_list = get_candidates_list([tree_first_guess], [tree_first_response_pattern])
print(f'list of possible solutions ({len(tree_first_candidates_list):,}): {conjunctify_nouns(tree_first_candidates_list)}')
tree_second_guess = get_word_guess(response_patterns_df, tree_first_candidates_list, guesses_list=[tree_first_guess])
print(f'SECOND MOVE: {tree_second_guess}')

list of possible solutions (653): bicep, bided, bidet, biked, bikie, binge, biome, biped, bitte, bocce, boche, boded, bodge, boeuf, bogey, bogie, bogue, boite, boked, bokeh, bombe, bonce, boned, boney, bonie, bonne, bonze, booed, booze, botte, bouge, bowed, bowet, bowie, bowne, boxed, boxen, boyed, bubbe, budge, buffe, bunce, bunde, bunje, buteo, butte, byded, byked, check, cheek, cheep, chemo, cheth, chevy, chewy, chide, chief, chime, chine, chive, chode, choke, chute, chyme, cided, cimex, cited, civet, civie, codec, coded, coden, codex, cogie, cogue, cohen, cohoe, coked, combe, comet, comte, coned, coney, conge, conne, conte, cooed, cooee, cooey, cooze, coped, copen, coted, coude, coupe, coved, coven, covet, covey, cowed, coxed, coyed, cozed, cozen, cozey, cozie, ctene, cubeb, cubed, cumec, cunei, cutey, cutie, cuvee, diced, dicey, didie, diene, diked, dikey, dined, dinge, diode, dited, dived, dixie, dizen, dobie, dodge, doeth, dogey, dogie, domed, donee, donne, dooce, doped, dopey, 

In [24]:

tree_second_guess = 'saunt'
tree_second_response_pattern = 'xxYxY'
test_words_list = [tree_first_guess, tree_second_guess]
response_patterns_list = [tree_first_response_pattern, tree_second_response_pattern]
tree_second_candidates_list = get_candidates_list(test_words_list, response_patterns_list, tree_first_candidates_list)
print(f'list of possible solutions ({len(tree_second_candidates_list):,}): {conjunctify_nouns(tree_second_candidates_list)}')
tree_third_guess = get_word_guess(response_patterns_df, tree_second_candidates_list, guesses_list=test_words_list)
print(f'THIRD MOVE: {tree_third_guess}')

list of possible solutions (17): buteo, butte, cutey, cutie, muted, outed, quite, quote, quyte, togue, toque, tubed, tuffe, tupek, tuque, tutee, and uptie
THIRD MOVE: boite


In [25]:

tree_third_guess = 'hopes'
tree_third_response_pattern = 'xxxYx'
test_words_list = [tree_first_guess, tree_second_guess, tree_third_guess]
response_patterns_list = [tree_first_response_pattern, tree_second_response_pattern, tree_third_response_pattern]
tree_third_candidates_list = get_candidates_list(test_words_list, response_patterns_list, tree_second_candidates_list)
print(f'list of possible solutions ({len(tree_third_candidates_list):,}): {conjunctify_nouns(tree_third_candidates_list)}')
tree_fourth_guess = get_word_guess(response_patterns_df, tree_third_candidates_list, guesses_list=test_words_list)
print(f'FOURTH MOVE: {tree_fourth_guess}')

list of possible solutions (6): butte, cutie, quite, quyte, tuffe, and tuque
FOURTH MOVE: fisty


In [26]:

tree_fourth_guess = 'moile'
tree_fourth_response_pattern = 'xxGxG'
test_words_list = [tree_first_guess, tree_second_guess, tree_third_guess, tree_fourth_guess]
response_patterns_list = [tree_first_response_pattern, tree_second_response_pattern, tree_third_response_pattern, tree_fourth_response_pattern]
tree_fourth_candidates_list = get_candidates_list(test_words_list, response_patterns_list, tree_third_candidates_list)
print(f'list of possible solutions ({len(tree_fourth_candidates_list):,}): {conjunctify_nouns(tree_fourth_candidates_list)}')
tree_fifth_guess = get_word_guess(response_patterns_df, tree_fourth_candidates_list, guesses_list=test_words_list)
print(f'FIFTH MOVE: {tree_fifth_guess}')

list of possible solutions (1): quite
FIFTH MOVE: quite


In [27]:

tree_fifth_guess = 'sepia'
tree_fifth_response_pattern = 'xYxYx'
test_words_list = [tree_first_guess, tree_second_guess, tree_third_guess, tree_fourth_guess, tree_fifth_guess]
response_patterns_list = [tree_first_response_pattern, tree_second_response_pattern, tree_third_response_pattern, tree_fourth_response_pattern,
                          tree_fifth_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
tree_fifth_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                tree_fifth_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(tree_fifth_candidates_list)}): {conjunctify_nouns(sorted(tree_fifth_candidates_list))}')
tree_sixth_guess, max_score = get_best_word(tree_fifth_candidates_list)
print(f'SIXTH MOVE: {tree_sixth_guess}')

list of possible solutions: (1): quite
SIXTH MOVE: quite


In [28]:

tree_sixth_guess = 'quite'
tree_sixth_response_pattern = 'GGGGG'
test_words_list = [tree_first_guess, tree_second_guess, tree_third_guess, tree_fourth_guess, tree_fifth_guess, tree_sixth_guess]
response_patterns_list = [tree_first_response_pattern, tree_second_response_pattern, tree_third_response_pattern, tree_fourth_response_pattern,
                          tree_fifth_response_pattern, tree_sixth_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
tree_sixth_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                tree_sixth_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(tree_sixth_candidates_list)}): {conjunctify_nouns(sorted(tree_sixth_candidates_list))}')
tree_seventh_guess, max_score = get_best_word(tree_sixth_candidates_list)
print(f'SEVENTH MOVE: {tree_seventh_guess}')

list of possible solutions: (1): quite
SEVENTH MOVE: quite


In [22]:

tree_seventh_guess = 'gadjo'
tree_seventh_response_pattern = 'YYYxY'
test_words_list = [tree_first_guess, tree_second_guess, tree_third_guess, tree_fourth_guess, tree_fifth_guess, tree_sixth_guess, tree_seventh_guess]
response_patterns_list = [tree_first_response_pattern, tree_second_response_pattern, tree_third_response_pattern, tree_fourth_response_pattern,
                          tree_fifth_response_pattern, tree_sixth_response_pattern, tree_seventh_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
tree_seventh_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                tree_seventh_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(tree_seventh_candidates_list)}): {conjunctify_nouns(sorted(tree_seventh_candidates_list))}')
tree_eighth_guess, max_score = get_best_word(tree_seventh_candidates_list)
print(f'EIGHTH MOVE: {tree_eighth_guess}')

list of possible solutions: (1): dogma
EIGHTH MOVE: dogma



----
# Maximize Green Responses to Solve a Wordle

In [141]:

leaf_first_guess, max_score = get_best_word(words_list)
print(f'list of possible solutions: ({len(words_list):,})')
print(f'FIRST MOVE: {leaf_first_guess}')

list of possible solutions: (12,972)
FIRST MOVE: arose


In [63]:

leaf_first_guess = 'arise'
leaf_first_response_pattern = 'Yxxxx'
test_words_list = [leaf_first_guess]
response_patterns_list = [leaf_first_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
leaf_first_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                leaf_first_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(leaf_first_candidates_list)}): {conjunctify_nouns(sorted(leaf_first_candidates_list))}')
leaf_second_guess, max_score = get_best_word(leaf_first_candidates_list)
print(f'SECOND MOVE: {leaf_second_guess}')

list of possible solutions: (796): babka, baboo, babul, bacca, bacco, baccy, bacha, bacon, baddy, badly, baffy, baggy, bahut, bajan, baldy, balky, bally, balmy, baloo, balun, banak, banal, banco, banda, bandh, bandy, banjo, bantu, banty, banya, batch, baton, batta, battu, batty, baulk, bawdy, bawty, bayou, bazoo, bhang, bhuna, black, blady, blaff, bland, blank, blatt, blaud, blawn, bloat, bobac, bobak, bocca, bogan, bonza, booay, bowat, boxla, boyau, boyla, bubal, bubba, buffa, bulla, bunya, bwana, bylaw, byway, cabal, cabby, cabob, caboc, cacao, cacky, caddy, cadgy, cagot, cahow, cajon, cajun, calla, calmy, calpa, calyx, caman, campo, campy, canal, candy, canna, canny, canon, canto, canty, capon, capot, capul, caput, catch, catty, cauda, cauld, caulk, caxon, chack, chaco, chado, chaff, chaft, chalk, champ, chana, chang, chank, chant, chapt, chawk, chaya, chola, chota, chufa, clach, clack, clamp, clang, clank, clapt, claut, cloak, cloam, coach, coact, coady, coala, coaly, coapt, cobza,

In [64]:

leaf_second_guess = 'slipt'
leaf_second_response_pattern = 'xxxxx'
test_words_list = [leaf_first_guess, leaf_second_guess]
response_patterns_list = [leaf_first_response_pattern, leaf_second_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
leaf_second_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                leaf_second_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(leaf_second_candidates_list)}): {conjunctify_nouns(sorted(leaf_second_candidates_list))}')
leaf_third_guess, max_score = get_best_word(leaf_second_candidates_list)
print(f'THIRD MOVE: {leaf_third_guess}')

list of possible solutions: (310): babka, baboo, bacca, bacco, baccy, bacha, bacon, baddy, baffy, baggy, bajan, banak, banco, banda, bandh, bandy, banjo, banya, bawdy, bayou, bazoo, bhang, bhuna, bobac, bobak, bocca, bogan, bonza, booay, boyau, bubba, buffa, bunya, bwana, byway, cabby, cabob, caboc, cacao, cacky, caddy, cadgy, cahow, cajon, cajun, caman, candy, canna, canny, canon, cauda, caxon, chack, chaco, chado, chaff, chana, chang, chank, chawk, chaya, chufa, coach, coady, cobza, cocoa, cohab, comma, conga, cowan, coyau, cyano, cycad, dabba, dacha, dadah, daddy, daffy, dagga, daggy, daman, dancy, dandy, danny, dauby, dawah, dayan, daych, dhaba, dogan, dogma, donah, donga, donna, doona, douma, downa, dukka, dumka, dunam, dwang, dwaum, fabby, faddy, faffy, faggy, fancy, fanga, fango, fanny, fanon, fanum, faugh, fauna, fawny, foamy, fonda, gabba, gabby, gadjo, gamay, gamba, gambo, gamma, gammy, ganch, gandy, ganja, ganof, gauch, gaucy, gaudy, gaumy, gauzy, gawcy, gawky, gazon, gazoo,

In [65]:

leaf_third_guess = 'smith'
leaf_third_response_pattern = 'xYxxx'
test_words_list = [leaf_first_guess, leaf_second_guess, leaf_third_guess]
response_patterns_list = [leaf_first_response_pattern, leaf_second_response_pattern, leaf_third_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
leaf_third_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                leaf_third_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(leaf_third_candidates_list)}): {conjunctify_nouns(sorted(leaf_third_candidates_list))}')
leaf_fourth_guess, max_score = get_best_word(leaf_third_candidates_list)
print(f'FOURTH MOVE: {leaf_fourth_guess}')

list of possible solutions: (59): caman, comma, daman, dogma, douma, dumka, dunam, dwaum, fanum, foamy, gamay, gamba, gambo, gamma, gammy, gaumy, gumma, jambo, jambu, jammy, jamon, kaama, kokam, macaw, macon, madam, magma, mamba, mambo, mamma, mammy, manga, mango, mangy, manky, manna, mauby, maund, mauzy, mawky, mayan, momma, monad, mugga, munga, muzak, myoma, namma, ngoma, nomad, oakum, occam, undam, unjam, unman, woman, yamun, zaman, and zambo
FOURTH MOVE: mango


In [67]:

leaf_fourth_guess = 'stink'
leaf_fourth_response_pattern = 'xxxxx'
test_words_list = [leaf_first_guess, leaf_second_guess, leaf_third_guess, leaf_fourth_guess]
response_patterns_list = [leaf_first_response_pattern, leaf_second_response_pattern, leaf_third_response_pattern, leaf_fourth_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
leaf_fourth_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                leaf_fourth_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(leaf_fourth_candidates_list)}): {conjunctify_nouns(sorted(leaf_fourth_candidates_list))}')
leaf_fifth_guess, max_score = get_best_word(leaf_fourth_candidates_list)
print(f'FIFTH MOVE: {leaf_fifth_guess}')

list of possible solutions: (29): comma, dogma, douma, dwaum, foamy, gamay, gamba, gambo, gamma, gammy, gaumy, gumma, jambo, jambu, jammy, macaw, madam, magma, mamba, mambo, mamma, mammy, mauby, mauzy, momma, mugga, myoma, occam, and zambo
FIFTH MOVE: gambo


In [68]:

leaf_fifth_guess = 'sulfo'
leaf_fifth_response_pattern = 'xxxxY'
test_words_list = [leaf_first_guess, leaf_second_guess, leaf_third_guess, leaf_fourth_guess, leaf_fifth_guess]
response_patterns_list = [leaf_first_response_pattern, leaf_second_response_pattern, leaf_third_response_pattern, leaf_fourth_response_pattern,
                          leaf_fifth_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
leaf_fifth_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                leaf_fifth_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(leaf_fifth_candidates_list)}): {conjunctify_nouns(sorted(leaf_fifth_candidates_list))}')
leaf_sixth_guess, max_score = get_best_word(leaf_fifth_candidates_list)
print(f'SIXTH MOVE: {leaf_sixth_guess}')

list of possible solutions: (5): comma, dogma, momma, myoma, and occam
SIXTH MOVE: comma


In [69]:

leaf_sixth_guess = 'scowl'
leaf_sixth_response_pattern = 'xxYxx'
test_words_list = [leaf_first_guess, leaf_second_guess, leaf_third_guess, leaf_fourth_guess, leaf_fifth_guess, leaf_sixth_guess]
response_patterns_list = [leaf_first_response_pattern, leaf_second_response_pattern, leaf_third_response_pattern, leaf_fourth_response_pattern,
                          leaf_fifth_response_pattern, leaf_sixth_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
leaf_sixth_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                leaf_sixth_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(leaf_sixth_candidates_list)}): {conjunctify_nouns(sorted(leaf_sixth_candidates_list))}')
leaf_seventh_guess, max_score = get_best_word(leaf_sixth_candidates_list)
print(f'SEVENTH MOVE: {leaf_seventh_guess}')

list of possible solutions: (2): dogma and momma
SEVENTH MOVE: dogma


In [70]:

leaf_seventh_guess = 'scold'
leaf_seventh_response_pattern = 'xxYxY'
test_words_list = [leaf_first_guess, leaf_second_guess, leaf_third_guess, leaf_fourth_guess, leaf_fifth_guess, leaf_sixth_guess, leaf_seventh_guess]
response_patterns_list = [leaf_first_response_pattern, leaf_second_response_pattern, leaf_third_response_pattern, leaf_fourth_response_pattern,
                          leaf_fifth_response_pattern, leaf_sixth_response_pattern, leaf_seventh_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
leaf_seventh_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                leaf_seventh_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(leaf_seventh_candidates_list)}): {conjunctify_nouns(sorted(leaf_seventh_candidates_list))}')
leaf_eighth_guess, max_score = get_best_word(leaf_seventh_candidates_list)
print(f'EIGHTH MOVE: {leaf_eighth_guess}')

list of possible solutions: (1): dogma
EIGHTH MOVE: dogma


In [71]:

leaf_eighth_guess = 'quoth'
leaf_eighth_response_pattern = 'xxYxx'
test_words_list = [leaf_first_guess, leaf_second_guess, leaf_third_guess, leaf_fourth_guess, leaf_fifth_guess, leaf_sixth_guess, leaf_seventh_guess, leaf_eighth_guess]
response_patterns_list = [leaf_first_response_pattern, leaf_second_response_pattern, leaf_third_response_pattern, leaf_fourth_response_pattern,
                          leaf_fifth_response_pattern, leaf_sixth_response_pattern, leaf_seventh_response_pattern, leaf_eighth_response_pattern]
yellows_list = get_yellows_list(test_words_list, response_patterns_list)
greys_list = get_greys_list(test_words_list, response_patterns_list)
leaf_eighth_candidates_list = []
for word_str in words_list:
    if all(map(lambda x: x in word_str, yellows_list)):
        if all(map(lambda x: x not in word_str, greys_list)):
            if is_greened(word_str, test_words_list, response_patterns_list):
                leaf_eighth_candidates_list.append(word_str)
print(f'list of possible solutions: ({len(leaf_eighth_candidates_list)}): {conjunctify_nouns(sorted(leaf_eighth_candidates_list))}')
leaf_ninth_guess, max_score = get_best_word(leaf_eighth_candidates_list)
print(f'NINTH MOVE: {leaf_ninth_guess}')

list of possible solutions: (1): dogma
NINTH MOVE: dogma



----
## Display the Difference

In [197]:

def patternize_guess(word_str, pattern_str):
    pre_str = '<span style="font-family:Courier;font-size:3em;text-transform:uppercase;font-variant-numeric:tabular-nums lining-nums;background-color:'
    in_str = ';">'
    post_str = '</span>'
    patternized_str = ''
    for letter_str, letter_pattern in zip(word_str, pattern_str):
        patternized_str += pre_str
        if letter_pattern == 'G':
            patternized_str += 'green'
        elif letter_pattern == 'Y':
            patternized_str += 'yellow'
        else:
            patternized_str += 'grey'
        patternized_str += in_str + letter_str + post_str
    
    return patternized_str

In [198]:

from num2words import num2words

def get_tr(ordinal, leaf_count=None, tree_count=None):
    ordinal_str = num2words(ordinal, lang='en', to='ordinal')
    previous_ordinal_str = num2words(ordinal-1, lang='en', to='ordinal')
    if leaf_count is None:
        leaf_count = eval(f'len(leaf_{previous_ordinal_str}_candidates_list)')
    leaf_guess = eval(f'leaf_{ordinal_str}_guess')
    leaf_pattern = eval(f'leaf_{ordinal_str}_response_pattern')
    if tree_count is None:
        tree_count = eval(f'len(tree_{previous_ordinal_str}_candidates_list)')
    tree_guess = eval(f'tree_{ordinal_str}_guess')
    tree_pattern = eval(f'tree_{ordinal_str}_response_pattern')
    html_str = f'''<tr>
        <td>{ordinal_str.title()}</td>
        <td>{leaf_count:,}</td>
        <td>{patternize_guess(leaf_guess, leaf_pattern)}</td>
        <td>{tree_count:,}</td>
        <td>{patternize_guess(tree_guess, tree_pattern)}</td>
    </tr>'''
    
    return html_str

In [199]:

from IPython.display import HTML, display

HTML(f'''<table>
    <tr>
        <th rowspan="2">Move</th>
        <th colspan="2" style="text-align:center">Green Maximizer</th>
        <th colspan="2" style="text-align:center">Branch Minimizer</th>
    </tr>
    <tr>
        <th style="text-align:center">Solutions</th>
        <th style="text-align:center">Guess</th>
        <th style="text-align:center">Solutions</th>
        <th style="text-align:center">Guess</th>
    </tr>
    {get_tr(1, leaf_count=len(words_list), tree_count=len(words_list))}
    {get_tr(2)}
    {get_tr(3)}
    {get_tr(4)}
    {get_tr(5)}
    {get_tr(6)}
</table>''')


----
# Explore the Data

In [94]:

response_patterns_set = set(['xGxxx', 'xGxYx', 'xYYYY'])
# mask_series = False
# for test_word, response_pattern in zip(test_words_list, response_patterns_list):
#     mask_series = mask_series | ((response_patterns_df.test_word == test_word) & (response_patterns_df.response_pattern == response_pattern))
mask_series = response_patterns_df.response_pattern.isin(response_patterns_set)
counts_df = response_patterns_df[mask_series].groupby('target_word').count()

In [95]:

counts_df.sort_values('response_pattern', ascending=False).head(5)

,test_word,response_pattern
target_word,,
jaffa,1862,1862
qajaq,1816,1816
mamma,1751,1751
baboo,1676,1676
kazoo,1667,1667


In [ ]:

if s.pickle_exists('english_letter_frequencies_df'):
    letter_proportions_df = s.load_object('english_letter_frequencies_df')
else:
    
    # Get proportions from word list
    import collections

    letters_list = []
    for word_str in words_list:
        letters_list += list(word_str)
    counter = collections.Counter(letters_list)
    letter_proportions_df = pd.DataFrame([{'letter_char': x, 'letter_count': y} for x, y in dict(counter).items()])
    min_count = letter_proportions_df.letter_count.min()
    letter_proportions_df['proportion'] = letter_proportions_df.letter_count.map(lambda x: x/min_count)
    s.store_objects(english_letter_frequencies_df=letter_proportions_df)

In [ ]:

import random

random.sample(words_list, 5)

In [ ]:

letter_proportions_df.sample(5)

In [ ]:

mask_series = (letter_proportions_df.letter_char == 'e')
letter_proportions_df[mask_series]

In [ ]:

letter_proportions_df.sort_values('proportion')


----

In [ ]:

pattern_groupby = response_patterns_df.groupby('response_pattern').count()

In [ ]:

pattern_groupby.sort_values('test_word', ascending=False).head(5)

In [ ]:

mask_series = response_patterns_df.target_word.isin(candidates_list)
pattern_groupby = response_patterns_df[mask_series].groupby('response_pattern').count().sort_values('test_word', ascending=False)
pattern_groupby.head(5)

In [ ]:

pattern_groupby.tail(5)

In [ ]:

mask_series = response_patterns_df.target_word.isin(candidates_list) & (response_patterns_df.response_pattern.isin(['GGYGY', 'GYGGY', 'GYxGG']))
response_patterns_df[mask_series]

In [ ]:

mask_series = response_patterns_df.test_word.isin(['arise'])
pattern_groupby = response_patterns_df[mask_series].groupby('response_pattern').count().sort_values('test_word', ascending=False)
pattern_groupby.head(5)

In [ ]:

test_list = ['arise']
pattern_groupby.loc[test_list]

In [ ]:

# Get words from GloVe: Global Vectors for Word Representation
import re

words_list = []
file_path = r'D:\Documents\GitHub\job-hunting\data\6B\glove.6B.300d.txt'
az_regex = re.compile('[^a-z]')
with open(file_path, encoding='utf8') as infile:
    for line in infile:
        word_str = line.split()[0]
        if (len(word_str) == 5) and not az_regex.search(word_str):
            words_list.append(word_str)
words_list.remove('aerio')
words_list.remove('ioane')
words_list.remove('eonia')
words_list.remove('tabou')
words_list.remove('tamou')
words_list.remove('tarom')
words_list.remove('tarmo')
words_list.remove('marot')
words_list.remove('marto')

In [ ]:

def build_lists(candidate_word, colors_list, greys_list=None, yellows_list=None, greens_list=None):
    if greys_list is None: greys_list = []
    if yellows_list is None: yellows_list = []
    if greens_list is None: greens_list = []
    for color, index in zip(range(len(candidate_word)), colors_list):
        if color == 'grey':
            greys_list.append(candidate_word[index])
        elif color == 'yellow':
            yellows_list.append(candidate_word[index])
        elif color == 'green':
            greens_list.append(f"(candidate_word[{index}] == '{candidate_word[index]}')")
    
    return greys_list, yellows_list, greens_list

In [ ]:

def get_largest_equivalence_class(candidates_list, base_greys_list=None, base_yellows_list=None, base_greens_list=None):
    if base_greys_list is None: base_greys_list = []
    if base_yellows_list is None: base_yellows_list = []
    if base_greens_list is None: base_greens_list = []
    max_possibles_list = []
    max_candidate_word = ''
    for candidate_word in candidates_list:
        possibles_list = []
        for color1 in ['grey', 'yellow', 'green']:
            for color2 in ['grey', 'yellow', 'green']:
                for color3 in ['grey', 'yellow', 'green']:
                    for color4 in ['grey', 'yellow', 'green']:
                        for color5 in ['grey', 'yellow', 'green']:
                            colors_list = [color1, color2, color3, color4, color5]
                            greys_list, yellows_list, greens_list = build_lists(candidate_word, colors_list,
                                                                                base_greys_list, base_yellows_list, base_greens_list)
                            if all(map(lambda x: eval(x), greens_list)):
                                if all(map(lambda x: x in word_str, yellows_list)):
                                    if all(map(lambda x: x not in word_str, greys_list)):
                                        possibles_list.append(candidate_word)

In [ ]:

for word_str in candidates_list:
    greys_list = ['a', 'i', 's']
    yellows_list = ['r']
    greens_list = ["(candidate_word[4] == 'e')"]
    max_possibles_list = get_largest_equivalence_class(word_str, base_greys_list=greys_list, base_yellows_list=yellows_list, base_greens_list=greens_list)

In [ ]:

if s.pickle_exists('english_word_frequencies_df'):
    word_proportions_df = s.load_object('english_word_frequencies_df')
else:
    
    # Get words from the Wordle JavaScript
    if s.pickle_exists('wordle_words_list'):
        words_list = s.load_object('wordle_words_list')
    else:

        # Get words from the Wordle JavaScript
        import requests

        link = 'https://www.powerlanguage.co.uk/wordle/main.db1931a8.js'
        f = requests.get(link)
        commands_list = f.text.split(';')
        list_str = commands_list[331].split(']')[0].split('[')[1]
        words_list = eval(f'[{list_str}]')
        s.store_objects(wordle_words_list=words_list)
        
    rows_list = [{'word_str': word_str, 'proportion_score': get_proportion_score(word_str)} for word_str in words_list]
    word_proportions_df = pd.DataFrame(rows_list)
    s.store_objects(english_word_frequencies_df=word_proportions_df)

In [ ]:

word_proportions_df.sample(5)

In [ ]:

proportion_score_dict = word_proportions_df.set_index('word_str').proportion_score.to_dict()